# Data Extraction from Semi-Structured Files

## Install Libraries

In [1]:
!pip3 install -U openpyxl pandas

     |████████████████████████████████| 11.7 MB 7.6 MB/s eta 0:00:01


## Extract Semi-Structured Data - Spreadsheet

In [49]:
import pandas as pd
import numpy as np
import datetime
import glob
from IPython.display import display
pd.options.display.max_columns = None

In [12]:
SEMI_STRUCTURED_EXCEL = "test_data/semi_structured_spreadsheet.xlsx"
SHEET_NAME = "data"

In [13]:
def is_day(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False
    
def is_valid_date(dt_str):
    try:
        dt = datetime.datetime.strptime(dt_str, '%Y-%m-%d')
        return True
    except ValueError:
        return False
    
def get_all_sheets(file_name):
       excel = pd.ExcelFile(file_name, engine="openpyxl")
       return excel.sheet_names

In [93]:
# Read the excel sheet, skip 3 rows, 2 headers
df = pd.read_excel(SEMI_STRUCTURED_EXCEL, engine="openpyxl", sheet_name=SHEET_NAME, header=[0,1], skiprows=3)

#display(df.head(20))

# Drop the summary fields
df.drop(columns=["Total Sum of Units", "Total Sum of Order Amount"], level=0, inplace=True)

# Flatten columns
df.columns = [' '.join(col).strip() for col in df.columns.values]

# Rename columns
df.rename(columns = {"Unnamed: 0_level_0 Country": "Country", "Unnamed: 1_level_0 Salesperson": "Salesperson"}, inplace=True)

# Forward fill for Country and Salesperson
df.loc[:, ["Country","Salesperson"]] = df.loc[:, ["Country", "Salesperson"]].ffill()

# Drop the result rows
df = df[~df.Country.str.contains("Result")]

# Fill NaN with 0
df.fillna(0, inplace=True)

# Convert columns to rows
df = df.melt(id_vars=["Country", "Salesperson"], 
        var_name="Type", 
        value_name="Value")

# Create new column for year and type
df['Year'] = df.Type.str.slice(start=0, stop=5)
df['Type'] = df.Type.str.slice(start=5)

# Change the description
df['Type'].replace({"Sum of Units": "Units", "Sum of Order Amount": "Order Amount"}, inplace=True)

# Pivot it
df = df.pivot(index=["Country", "Salesperson", "Year"], columns=["Type"],values="Value")

# Reset the index
df.reset_index(inplace=True)
display(df.head(100))

# Save to CSV
df.to_csv(path_or_buf="test_data/structured_data.csv", index=False)

Type,Country,Salesperson,Year,Order Amount,Units
0,UK,Bromley,2011,24756.89,232.0
1,UK,Bromley,2012,40396.64,228.0
2,UK,Bromley,2013,9894.51,73.0
3,UK,Coghill,2011,4029.25,81.0
4,UK,Coghill,2012,4657.11,39.0
5,UK,Coghill,2013,0.00,0.0
6,UK,Farnham,2011,14055.87,170.0
7,UK,Farnham,2012,5892.65,44.0
8,UK,Farnham,2013,2560.40,17.0
9,UK,Gillingham,2011,40826.37,397.0
